# Flightops - Data Acquisition, Loading and Pre-processing

## Extracting Data from BTS
Flight data for 2018–2019 is sourced from the BTS TranStats repository using an automated downloader script:
- `download_bts_ontime.py` fetches monthly On-Time Performance files and extracts them into the project directory.
- Run this command from the repo root to download the necessary files: *python download_bts_ontime.py*

## Load, and Clean Data

Data cleaning is one of the most critical (and usually most tedious) part of data analysis. But the effort is always worth it because, as the old saying goes, **garbage in, garbage out!**

environment: conda activate datasci311

### Set stable root directory for relative paths in VSCode

VS Code + Jupyter has notoriously unstable CWDs because it is flexible in where it runs notebooks from (e.g. the workspace root, a previously cached directory, a kernel opened *before* the repo was opened...). It's great that VSC has this flexibility, but it presents a weakness when making repos available in CI/GitHub. I've removed the ambiguity by controlling the CWD to make relative paths stable.

In [2]:
from pathlib import Path
import pandas as pd

# Establish repo root (notebooks live one level down)
REPO_ROOT = Path.cwd().parent

DATA_DIR = REPO_ROOT / "data/raw/csv"


### Load and Concatenate Files

After extracting the required files from BTS, we'll load and combine them into one dataframe.

In [ ]:
files = sorted(DATA_DIR.glob("*.csv"))
print(f"Found {len(files)} CSV files.") # used for visibility when debugging cwd issues

# Specify columns to load to save memory, reducing columns loaded from 110 to 30 and dataframe memory usage from ~24.8 GB to ~11.1 GB
usecols = [
    # Date / identifiers
    "Year",
    "Month",
    "DayofMonth",
    "DayOfWeek",
    "FlightDate",
    "Reporting_Airline",
    "IATA_CODE_Reporting_Airline",
    "Flight_Number_Reporting_Airline",
    "Tail_Number",

    # Origin / destination
    "Origin",
    "OriginCityName",
    "OriginState",
    "Dest",
    "DestCityName",
    "DestState",

    # Scheduled vs actual times
    "CRSDepTime",
    "CRSArrTime",
    "DepTime",
    "ArrTime",

    # Delay metrics
    "DepDelay",
    "DepDelayMinutes",
    "DepDel15",
    "ArrDelay",
    "ArrDelayMinutes",
    "ArrDel15",

    # Status flags
    "Cancelled",
    "Diverted",
    "CancellationCode",

    # Distance / throughput
    "Distance",
    "Flights",
]

# Adding a dtype map to reduce memory usage further
dtypes = {
    "Year": "int16",
    "Month": "int8",
    "DayofMonth": "int8",
    "DayOfWeek": "int8",
    "Reporting_Airline": "category",
    "IATA_CODE_Reporting_Airline": "category",
    "Tail_Number": "category",
    "Flight_Number_Reporting_Airline": "int32",
    "Origin": "category",
    "OriginCityName": "category",
    "OriginState": "category",
    "Dest": "category",
    "DestCityName": "category",
    "DestState": "category",
    "CRSDepTime": "int32",
    "CRSArrTime": "int32",
    "DepTime": "float32",
    "ArrTime": "float32",
    "DepDelay": "float32",
    "DepDelayMinutes": "float32",
    "DepDel15": "Int8",
    "ArrDelay": "float32",
    "ArrDelayMinutes": "float32",
    "ArrDel15": "Int8",
    "Cancelled": "Int8",
    "Diverted": "Int8",
    "CancellationCode": "category",
    "Distance": "float32",
    "Flights": "Int8",
}

dfs = [pd.read_csv(f, low_memory=False, usecols=usecols, dtype=dtypes) for f in files]

flights = pd.concat(dfs, ignore_index=True)


Found 24 CSV files.


In [14]:
len(flights), flights.memory_usage(deep=True).sum() / (1024**3)


(14628232, np.float64(8.14171683229506))

#### Find TranStats terminology at:
[BTS Table Info](https://www.transtats.bts.gov/TableInfo.asp?gnoyr_VQ=FGJ&QO_fu146_anzr=b0-gvzr&V0s1_b0yB=D)

### Data Cleaning

BTS column names are in all caps, with mixed formatting, so we'll strip out white space and special characters and replace spaces and hyphens with underscores for standard Python-friendly formatting.

Additionally, these BTS tables report on cancelled and diverted flights. This data will distort KPIs like average delay and on-time rates because they do not have meaningful arrival delay values. Therefore, we will filter them out.


In [15]:
# Standardize column names

flights.columns = (
    flights.columns.str.strip()  # Remove leading/trailing whitespace
        .str.lower()  # Lowercase all characters
        .str.replace(" ", "_")  # Replace spaces with underscores
)
print(flights.columns.tolist())

['year', 'month', 'dayofmonth', 'dayofweek', 'flightdate', 'reporting_airline', 'iata_code_reporting_airline', 'tail_number', 'flight_number_reporting_airline', 'origin', 'origincityname', 'originstate', 'dest', 'destcityname', 'deststate', 'crsdeptime', 'deptime', 'depdelay', 'depdelayminutes', 'depdel15', 'crsarrtime', 'arrtime', 'arrdelay', 'arrdelayminutes', 'arrdel15', 'cancelled', 'cancellationcode', 'diverted', 'flights', 'distance']


In [ ]:
# Sanity check: can we treat missing flags as 0/1?

flights["cancelled"].value_counts(dropna=False).head()
flights["diverted"].value_counts(dropna=False).head()


In [ ]:
# Fill missing flags as 0/1, removing any NaNs that may interfere with filtering

flag_cols = ["cancelled", "diverted", "depdel15", "arrdel15", "flights"]
for c in flag_cols:
    flights[c] = flights[c].fillna(0).astype("int8")


In [16]:
# Filter Cancelled/Diverted Flights

flights_clean = flights[
    (flights['cancelled'] == 0) &
    (flights['diverted'] == 0)
].copy()

print(f"Rows before cleaning: {len(flights)}")
print(f"Rows after cleaning: {len(flights_clean)}")

Rows before cleaning: 14628232
Rows after cleaning: 14340046


<mark>personal note</mark>

.copy() uses a lot of RAM. Flights currently uses ~8G memory, which pushes to 16G when executing above code. If more issues arise due to memory overhead (other apps open and using RAM), then use this code to filter with a mask, selected only needed columns, before you copy:

mask = (flights["cancelled"] == 0) & (flights["diverted"] == 0)
flights_clean = flights.loc[mask].copy()

print(f"Rows before cleaning: {len(flights):,}")
print(f"Rows after cleaning:  {len(flights_clean):,}")


In [ ]:
# Delete flights dataframe to free up memory (and prevent crashes during modeling)
del flights
import gc
gc.collect()

9